## Query the Knowledge Graph

### Import Libraries

In [15]:
import os
from dotenv import load_dotenv

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

### Load Environment Variables and Set Constants

In [17]:
load_dotenv()

# Constants
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
LLM_TYPE = os.getenv("LLM_TYPE", "ollama")  # Default to 'ollama' if not set
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.1")  # Default to 'llama3.1' if not set

os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

In [18]:
print(f"Using LLM type: {LLM_TYPE}")

Using LLM type: google


### Establish Connection to Neo4j

In [19]:
graph = Neo4jGraph(
  url=NEO4J_URI,
  username=NEO4J_USERNAME,
  password=NEO4J_PASSWORD,
)

### Chain for Extracting Entities

In [20]:
if LLM_TYPE == "ollama":
  print("Using Ollama LLM")
  llm = ChatOllama(
    model="llama3.1",
    temperature=0,
  )
else:
  print("Using Google Gemini LLM")
  llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0,
  )

Using Google Gemini LLM


In [21]:
class Entities(BaseModel):
  """Identifying informatoin about entities"""

  names: list[str] = Field(
    ...,
    description="All the person, organization, or business entities that appears in the text",
  )

prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are extracting organization and person entities from the text"
    ),
    (
      "human",
      "Use the given format to extract information from the following "
      "input: {question}",
    )
  ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

### Query the Graph Database

In [34]:
def generate_full_text_query(input: str) -> str:
  words = [el for el in remove_lucene_chars(input).split() if el]
  if not words:
    return ""
  full_text_query = " AND ".join([f"{word}~2" for word in words])
  print(f"Generated Query: {full_text_query}")
  return full_text_query.strip()

# Fulltext index query
def graph_retriever(question: str) -> str:
  """
  Collects the neighborhood of entities mentioned
  in the question
  """
  result = ""
  entities = entity_chain.invoke({"question": question})
  print(entities)
  for entity in entities.names:
    response = graph.query(
      """CALL db.index.fulltext.queryNodes('entity', $query, {limit: 2})
      YIELD node, score
      CALL {
        WITH node
        MATCH (node)-[r:!MENTIONS]->(neighbor)
        RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
        UNION ALL
        WITH node
        MATCH (node)<-[r:!MENTIONS]-(neighbor)
        RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
      }
      RETURN output LIMIT 20
      """,
      {"query": entity},
    )
    result += "\n".join([el['output'] for el in response])
  return result

In [35]:
retrieved = graph_retriever("กฎหมายที่เกี่ยวกับภาษีเงินได้มีอะไรบ้าง?")
print(retrieved)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 7, offset: 94} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit: 2})\n      YIELD node, score\n      CALL {\n        WITH node\n        MATCH (node)-[r:!MENTIONS]->(neighbor)\n        RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n        UNION ALL\n        WITH node\n        MATCH (node)<-[r:!MENTIONS]-(neighbor)\n        RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output\n      }\n      RETURN output LIMIT 20\n      "


names=['ภาษีเงินได้']
ภาษีเงินได้ - RELATED_TO -> ภาษี
ภาษีเงินได้ - RELATED_TO -> มาตรฐานการบัญชี
ภาษีเงินได้ - RELATED_TO -> หนี้สิน
ภาษีเงินได้ - RELATED_TO -> รายได้
ภาษีเงินได้ - RELATED_TO -> ค่าใช้จ่าย
ภาษีเงินได้ - RELATED_TO -> บริษัทใหญ่
ภาษีเงินได้ - RELATED_TO -> การลงทุน
ภาษีเงินได้ - RELATED_TO -> งวดปัจจุบัน
ภาษีเงินได้ - RELATED_TO -> สินทรัพย์ภาษีเงินได้
ภาษีเงินได้ - RELATED_TO -> การตัดบัญชี
ภาษีเงินได้ - RELATED_TO -> เกณฑ์เงินสด
ภาษีเงินได้ - RELATED_TO -> งวดที่ผ่านมา
ภาษีเงินได้ - RELATED_TO -> หนี้สนิ
ภาษีเงินได้ - RELATED_TO -> ผลต่างระหว่างมูลค่า ตามบัญชีของสิน ทรัพย์กับฐานภาษีของสิน ทรัพย์
ภาษีเงินได้ - RELATED_TO -> ภาษีเงินได้รอการตัดบัญชี
ภาษีเงินได้ - RELATED_TO -> อัตราภาษี
ภาษีเงินได้ - RELATED_TO -> ค่าเสื่อมราคาสะสมทางภาษี
ภาษีเงินได้ - RELATED_TO -> เสียภาษี
ภาษีเงินได้ - RELATED_TO -> กาไรส่วนที่เกินต้นทุน
ภาษีเงินได้ - RELATED_TO -> การวัดมูลค่าอสังหาริมทรัพย์


### Vector Similarity Search

In [40]:
embeddings = OllamaEmbeddings(
  model="qllama/multilingual-e5-small"
)

vector_index = Neo4jVector.from_existing_graph(
  embedding=embeddings,
  search_type="hybrid",
  node_label="Document",
  text_node_properties=["text"],
  embedding_node_property="embedding",
)

vector_retriever = vector_index.as_retriever()

def vector_retrieve(question: str) -> str:
  """
  Using vector similarity to find relevant documents based on the entities
  in the question
  """
  result = ""
  entities = entity_chain.invoke({"question": question})
  for entity in entities.names:
    response = vector_retriever.invoke(entity)
    text = "\n".join([el.page_content for el in response])
    print(len(text), "characters retrieved")
    result += text

  return result

In [41]:
retrieved = vector_retrieve("กฎหมายที่เกี่ยวกับภาษีเงินได้มีอะไรบ้าง?").split("\n")
for el in retrieved:
  print(el)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

562 characters retrieved

text: 51ค

text: กิจการต้องเปิ ดเผยส่วนประกอบที่สาคัญของค่าใช้จ่ายภาษีเงินได้หรือรายได้ภาษีเงินได้แยกกัน
ส่วนประกอบของค่าใช้ จ่ายภาษีเงินได้ หรือรายได้ ภาษีเงินได้ อาจประกอบด้ วย
80.1 ค่าใช้ จ่าย (หรือรายได้ ) ภาษีเงินได้ ของงวดปัจจุบัน
80.2 รายการปรับปรุงภาษีเงินได้ ของงวดก่อนที่ รับรู้ในงวดปัจจุบัน
80.3 ค่าใช้ จ่าย (หรือรายได้ ) ภาษีเงินได้ รอการตัดบัญชี ของผลแตกต่างชั่วคราวที่ เกิดขึ้น
หรือที่ กลับรายการ
80.4 ค่าใช้ จ่าย (หรือรายได้ ) ภาษีเงินได้ รอการตัดบัญชีที่เกี่ ยวข้ องกับการเปลี่ ยนแปลง

text: สภาวิชาชีพบัญชี ในพระบรมราชูปถัมภ์

text: ขอบเขต
2

3


### Full Retriever

In [37]:
def full_retriever(question: str):
  graph_data = graph_retriever(question)
  vector_data = vector_retrieve(question) 
  final_data = f"""Graph data:
  {graph_data}
  vector data:
  {vector_data}
  """
  return final_data

### Final Chain

In [38]:
from langchain_core.runnables import RunnableLambda

template = """Answer the qyestion based only on the following context:
{context}

Question: {question}
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

def debug_inputs(inputs):
    context = inputs.get("context", "")
    contexts = context.split("\n")
    for el in contexts:
        print(el.strip())
    return inputs

debug_node = RunnableLambda(debug_inputs)

chain = (
  {
    "context": full_retriever,
    "question": RunnablePassthrough()
  }
  | prompt
  | llm 
  | StrOutputParser()
)

In [39]:
chain.invoke(input="กฎหมายที่เกี่ยวกับภาษีเงินได้มีอะไรบ้าง?")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 7, offset: 94} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit: 2})\n      YIELD node, score\n      CALL {\n        WITH node\n        MATCH (node)-[r:!MENTIONS]->(neighbor)\n        RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n        UNION ALL\n        WITH node\n        MATCH (node)<-[r:!MENTIONS]-(neighbor)\n        RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output\n      }\n      RETURN output LIMIT 20\n      "


names=['ภาษีเงินได้']


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'The provided text does not contain information about laws related to income tax. It only mentions concepts related to income tax such as expenses, income, and accounting standards.'